In [1]:
import sys
from flask import Flask, render_template,jsonify,request
from flask_cors import CORS
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
import numpy as np
import pandas as pd

In [2]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///db/inpatient.db"
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

In [3]:
Base = automap_base(metadata=db.metadata)
engine = db.get_engine()
Base.prepare(engine, reflect=True)
Inpatient = Base.classes.inpatient
Drg = Base.classes.drg

In [4]:
ids = '872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W/O MCC'
#, '193 - SIMPLE PNEUMONIA & PLEURISY W MCC', '194 - SIMPLE PNEUMONIA & PLEURISY W CC', '291 - HEART FAILURE & SHOCK W MCC', '292 - HEART FAILURE & SHOCK W CC', '392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC', '470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC', '690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC', '871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W MCC', '872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W/O MCC']

In [5]:
fils = []
response = db.session.query(Inpatient).filter_by(drg_definition = ids)
for r in response:
    rec = r.__dict__.copy()
    del rec['_sa_instance_state']
    fils.append(rec)

In [6]:
df = pd.DataFrame(fils)
df

,average_covered_charges,average_medicare_payments,average_total_payments,drg_definition,hrr_description,lat,lon,mhi,provider_city,provider_id,provider_name,provider_state,provider_street_address,provider_zip_code,total_discharges
0,25205.0,4311.0,5436.0,872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 H...,AL - Dothan,31.140065,-85.398289,38050,DOTHAN,010001,SOUTHEAST ALABAMA MEDICAL CENTER,AL,1108 ROSS CLARK CIRCLE,36301,123.0
1,12298.0,4632.0,5705.0,872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 H...,AL - Birmingham,34.196159,-86.196898,40086,BOAZ,010005,MARSHALL MEDICAL CENTER SOUTH,AL,2505 U S HIGHWAY 431 NORTH,35957,49.0
2,25470.0,4152.0,5473.0,872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 H...,AL - Birmingham,NaN,NaN,None,FLORENCE,010006,ELIZA COFFEE MEMORIAL HOSPITAL,AL,205 MARENGO STREET,35631,103.0
3,13891.0,4932.0,5714.0,872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 H...,AL - Dothan,31.281017,-86.256945,34064,OPP,010007,MIZELL MEMORIAL HOSPITAL,AL,702 N MAIN ST,36467,23.0
4,37792.0,5390.0,6269.0,872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 H...,AL - Birmingham,33.624485,-86.647907,50539,BIRMINGHAM,010011,ST VINCENT'S EAST,AL,50 MEDICAL PARK EAST DRIVE,35235,20.0
5,39337.0,4904.0,5778.0,872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 H...,AL - Birmingham,34.382994,-85.803307,41094,FORT PAYNE,010012,DEKALB REGIONAL MEDICAL CENTER,AL,200 MED CENTER DRIVE,35968,19.0
6,44306.0,4745.0,6330.0,872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 H...,AL - Birmingham,33.215623,-86.797656,67794,ALABASTER,010016,SHELBY BAPTIST MEDICAL CENTER,AL,1000 FIRST STREET NORTH,35007,43.0
7,20145.0,4362.0,5541.0,872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 H...,AL - Birmingham,34.755494,-87.701265,33869,SHEFFIELD,010019,HELEN KELLER MEMORIAL HOSPITAL,AL,1300 SOUTH MONTGOMERY AVENUE,35660,101.0
8,29563.0,5442.0,7143.0,872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 H...,AL - Montgomery,32.269243,-86.201494,42346,MONTGOMERY,010023,BAPTIST MEDICAL CENTER SOUTH,AL,2105 EAST SOUTH BOULEVARD,36116,38.0
9,29546.0,4583.0,5842.0,872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 H...,AL - Montgomery,32.352820,-86.257585,49912,MONTGOMERY,010024,JACKSON HOSPITAL & CLINIC INC,AL,1725 PINE STREET,36106,49.0


In [7]:
df.count()

average_covered_charges      2613
average_medicare_payments    2613
average_total_payments       2613
drg_definition               2613
hrr_description              2613
lat                          2473
lon                          2473
mhi                          2473
provider_city                2613
provider_id                  2613
provider_name                2613
provider_state               2613
provider_street_address      2613
provider_zip_code            2613
total_discharges             2613
dtype: int64

In [8]:
#the "total_discharges" column has both strings and floats in it. pd.to_numeric has not worked
types = []
for i in df['total_discharges']:
    types.append(type(i))

test = pd.Series(types)
test.value_counts()

<class 'float'>    2613
dtype: int64

In [9]:
#this is the function I made to try to correct it, but it has not worked yet
def sanitize(data):
    if type(data).__name__ != 'float':
        return (float( data.replace(",","")))
    else:
        return data
            

df['total_discharges'] = df['total_discharges'].apply(sanitize)
df.count()

average_covered_charges      2613
average_medicare_payments    2613
average_total_payments       2613
drg_definition               2613
hrr_description              2613
lat                          2473
lon                          2473
mhi                          2473
provider_city                2613
provider_id                  2613
provider_name                2613
provider_state               2613
provider_street_address      2613
provider_zip_code            2613
total_discharges             2613
dtype: int64

In [10]:
#the "total_discharges" column has both strings and floats in it. pd.to_numeric has not worked
types = []
for i in df['total_discharges']:
    types.append(type(i))

test = pd.Series(types)
test.value_counts()

<class 'float'>    2613
dtype: int64

In [11]:
summary = pd.pivot_table(df, index='hrr_description', values=['average_covered_charges', 'total_discharges'])
summary

,average_covered_charges,total_discharges
hrr_description,,
AK - Anchorage,40288.714286,36.000000
AL - Birmingham,32201.468750,42.843750
AL - Dothan,24240.200000,43.800000
AL - Huntsville,24866.000000,45.666667
AL - Mobile,23352.000000,38.700000
AL - Montgomery,31781.800000,35.800000
AL - Tuscaloosa,24889.000000,98.000000
AR - Fort Smith,19334.250000,40.000000
AR - Jonesboro,16109.000000,61.666667


In [12]:
json = summary.reset_index().to_json(orient="records")
print(json)

[{"hrr_description":"AK - Anchorage","average_covered_charges":40288.7142857143,"total_discharges":36.0},{"hrr_description":"AL - Birmingham","average_covered_charges":32201.46875,"total_discharges":42.84375},{"hrr_description":"AL - Dothan","average_covered_charges":24240.2,"total_discharges":43.8},{"hrr_description":"AL - Huntsville","average_covered_charges":24866.0,"total_discharges":45.6666666667},{"hrr_description":"AL - Mobile","average_covered_charges":23352.0,"total_discharges":38.7},{"hrr_description":"AL - Montgomery","average_covered_charges":31781.8,"total_discharges":35.8},{"hrr_description":"AL - Tuscaloosa","average_covered_charges":24889.0,"total_discharges":98.0},{"hrr_description":"AR - Fort Smith","average_covered_charges":19334.25,"total_discharges":40.0},{"hrr_description":"AR - Jonesboro","average_covered_charges":16109.0,"total_discharges":61.6666666667},{"hrr_description":"AR - Little Rock","average_covered_charges":23864.4705882353,"total_discharges":49.411764